In [1]:
from model_learning.examples.reward_model_multiagent_inference import *
from model_learning.examples.reward_model_multiagent_inference import OUTPUT_DIR

IMG_EXT = 'pdf'
FOLDER_NAME = f'team_trajs_{len(MODEL_ROLES)}models_{NUM_TRAJECTORIES}x{TRAJ_LENGTH}_{MODEL_RATIONALITY}'
case = 'gtoprd'
if case == 'gtoprd':
    MODEL_ROLES = ['GroundTruth', 'Opposite', 'Random']
elif case == 'rdtksc':
    MODEL_ROLES = ['Random', 'Task', 'Social']
elif case == 'gttk':
    MODEL_NAMES = ['GroundTruth', 'Task']
# FOLDER_NAME = f'team_trajs_{case}_{NUM_TRAJECTORIES}x{TRAJ_LENGTH}_{MODEL_RATIONALITY}'
# FOLDER_NAME = f'GT_team_trajs_{case}_{NUM_TRAJECTORIES}x{TRAJ_LENGTH}_{MODEL_RATIONALITY}'


TRAJ_DIR = os.path.join(OUTPUT_DIR, FOLDER_NAME)
print(TRAJ_DIR)
if not os.path.exists(TRAJ_DIR):
    os.mkdir(TRAJ_DIR)
LOAD = 1


/Users/haochen/Desktop/IRL/model-learning/model_learning/examples/output/examples/reward-model-multiagent/team_trajs_3models_16x25_1


In [2]:
if LOAD: # load collected traj history data file
    f = bz2.BZ2File(os.path.join(OUTPUT_DIR, f'{FOLDER_NAME}.pkl'), 'rb')
    team_trajs = pickle.load(f)
    f.close()
    print(team_trajs)
else:
    team_trajs = []

[[<model_learning.TeamStateinfoActionModelTuple object at 0x7fc26a517eb0>, <model_learning.TeamStateinfoActionModelTuple object at 0x7fc26a51a970>, <model_learning.TeamStateinfoActionModelTuple object at 0x7fc26a51ac10>, <model_learning.TeamStateinfoActionModelTuple object at 0x7fc26a51adf0>, <model_learning.TeamStateinfoActionModelTuple object at 0x7fc26a51a040>, <model_learning.TeamStateinfoActionModelTuple object at 0x7fc26e1dbaf0>, <model_learning.TeamStateinfoActionModelTuple object at 0x7fc26e1db970>, <model_learning.TeamStateinfoActionModelTuple object at 0x7fc26e1db730>, <model_learning.TeamStateinfoActionModelTuple object at 0x7fc26e1db370>, <model_learning.TeamStateinfoActionModelTuple object at 0x7fc26e1dbd00>, <model_learning.TeamStateinfoActionModelTuple object at 0x7fc26e1dbe80>, <model_learning.TeamStateinfoActionModelTuple object at 0x7fc26e1ee040>, <model_learning.TeamStateinfoActionModelTuple object at 0x7fc26e1ee160>, <model_learning.TeamStateinfoActionModelTuple obj

In [3]:
from model_learning.examples.reward_model_multiagent_inference import _get_fancy_name, _get_belief
from model_learning.util.plot import plot_evolution

team_models = {f'{agent_name}_{role_name}': 0 for role_name in MODEL_ROLES for agent_name in TEAM_AGENTS}
MODEL_NAMES = list(team_models.keys())

def get_team_model_evolution_data(team_trajs, show_info=False):
    model_evolve = []
    for traj_i, team_traj in enumerate(team_trajs):
        model_evolve_per_traj = []
        for step_i, tsam in enumerate(team_traj):
            if step_i > -1:
                _world_state = tsam.state
                if show_info:
                    print('==================')
                    print('Step', step_i)
                    [print(tsam.action[agent_name]) for agent_name in TEAM_AGENTS]
                    print(tsam.model_dist)
                model_dist = tsam.model_dist
                model_evolve_per_traj.append(list(model_dist.values()))
        model_evolve.append(model_evolve_per_traj)
    return np.array(model_evolve)

def get_agent_evolve_from(team_evolve, model_name):
    assert type(team_evolve) == np.ndarray
    assert model_name in MODEL_NAMES
    return team_evolve[:, :, MODEL_NAMES.index(model_name)]

if LOAD:
    team_probs = get_team_model_evolution_data(team_trajs)
    np.savez(os.path.join(TRAJ_DIR, f'team_trajs.npz'), team_probs=team_probs)

In [4]:
# import matplotlib.pyplot as plt
%matplotlib inline

team_probs = np.load(os.path.join(TRAJ_DIR, f'team_trajs.npz'))['team_probs']
# for traj_i in range(len(team_probs)):
#     for agent_name in TEAM_AGENTS:
#         model_idx = []
#         for model_i, name in enumerate(MODEL_NAMES):
#             if name.startswith(agent_name):
#                 model_idx.append(model_i)
#         agent_probs = team_probs[traj_i].T[model_idx]
#         agent_model_names = [_get_fancy_name(name) for n_i, name in enumerate(MODEL_NAMES) if n_i in model_idx]
#
#         plot_evolution(agent_probs, agent_model_names,
#                        f'Evolution of Model Inference in Trajectory {traj_i+1}', None,
#                        os.path.join(TRAJ_DIR, f'inference_{traj_i}_{agent_name}.pdf'), 'Time', 'Model Probability', True, [0,1])

In [5]:
from model_learning.util.plot_new import *
dummy_plotly()

print(TEAM_AGENTS)
print(TRAJ_LENGTH)

for agent_name in TEAM_AGENTS:
    dfs = []
    for e in range(NUM_TRAJECTORIES):
        dict = {}
        for model_i, model_name in enumerate(MODEL_NAMES):
            if model_name.startswith(agent_name):
                dict[model_name] = team_probs[e, :, model_i]
        df = pd.DataFrame.from_dict(dict, orient='index').transpose()
        df['Episode'] = e
        dfs.append(df)
    df = pd.concat(dfs)

    file_name = f'{agent_name}_inference_revolution'
    plot_timeseries(df, 'Model Inference',
                    output_img=os.path.join(TRAJ_DIR, f'{file_name}.{IMG_EXT}'), x_label='Steps',
                    y_label='Model Probability', var_label='Models', y_min=0, y_max=1, plot_mean=False,
                    group_by='Episode', average=True, width=800, height=300)

bb

['Medic', 'Explorer']
25


NameError: name 'bb' is not defined

In [ ]:
# for model_name in MODEL_NAMES:
#     agent_probs = get_agent_evolve_from(team_probs, model_name)
#     plot_evolution(agent_probs, [_get_fancy_name(model_name) for i in range(len(agent_probs))],
#                f'Evolution of Model Inference on {_get_fancy_name(model_name)}', None,
#                os.path.join(TRAJ_DIR, f'inference_{model_name}.pdf'), 'Time', 'Model Probability', True, [0,1])

In [ ]:
# import pandas as pd
# df = pd.read_csv('output/examples/multiagent-ToM-property-world/evo-rwd-weights.csv')
# rwd_weights = df.to_numpy()
# print(rwd_weights)
# plot_evolution(rwd_weights.T, ['dh', 'search', 'evacuate'] , 'Reward Parameters Evolution', None,
#               os.path.join('output/examples/multiagent-ToM-property-world', f'evo-rwd-weights.pdf'), 'Epoch', 'Weight', y_lim=[-0.25, 0.8])


In [ ]:

LEARNER_AGENT = 'medic'
CASE = 'rdtksc'
DATA_DIR = f'output/examples/multiagent-ToM-property-world/{CASE}_learner_{LEARNER_AGENT}'
file_name = 'evo-rwd-weights'
df = pd.read_csv(os.path.join(DATA_DIR, f'{file_name}.csv'))
df['Epoch'] = list(df.index)
print(df)

out_file_name = 'test-evo-rwd-weights'
plot_timeseries(df, 'Reward Learning',
                output_img=os.path.join(DATA_DIR, f'{out_file_name}.{IMG_EXT}'),
                x_label='Epoch', y_label='Weight', var_label='Reward Features',
                y_min=-0.1, y_max=0.5, plot_mean=False, group_by='Epoch',
                average=True, width=800, height=400)
#
# rwd_weights = df.to_numpy()
# print(rwd_weights)
# plot_evolution(rwd_weights.T, ['dh', 'search', 'evacuate'] , 'Reward Parameters Evolution', None,
#               os.path.join(DATA_DIR, f'evo-rwd-weights.pdf'), 'Epoch', 'Weight', y_lim=[-0.25, 0.8])

In [ ]:
LEARNER_AGENT = 'medic'
CASE = 'rdtksc'
DATA_DIR = f'output/examples/multiagent-ToM-property-world/{CASE}_learner_{LEARNER_AGENT}'
file_name = 'learner-theta'
df = pd.read_csv(os.path.join(DATA_DIR, f'{file_name}.csv'))

out_file_name = 'bar_learn_theta'
plot_bar(df, 'Learned Reward Weights',
         output_img=os.path.join(DATA_DIR, f'{out_file_name}.{IMG_EXT}'),
         x_label='Reward Features', y_label='Reward Weight',
         y_min=-0.25, y_max=0.75,width=600, height=400)